**Process of Data Collection & Conversion to .csv File**

In this process, we outline the code steps taken to collect the data and convert it to .csv files. 

In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import duckdb
import numpy as np
import time

These files were then read, cleaned, and manipulated in the phase 2 submission file. 

**Player Stats**
In this section after importing above, I collected the url and created a dataframe that collected all of the data from all of the seasons from the 2003-2004 season to the 2022-2023 season for the top player stats, ranking which players led during the season and what their stats were. I combined all of this data into one main data frame using a for loop. The data is originally filtered by average points per game for each player per season.

In [10]:
#Create a list of all of the seasons to collect data from
season_years = ["2003-04", "2004-05", "2005-06", "2006-07", "2007-08", "2008-09", "2009-10", "2010-11", \
    "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", \
    "2021-22","2022-23"]

#Intialize empty list for dataframes to be added to
dataframes = []

#Loop through the season years and 
for season_year in season_years:

    #Add {season_year} into the api url to change each iteration
    url = f"https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season={season_year}&SeasonType=Regular%20Season&StatCategory=PTS"
    response = requests.get(url).json()

    table_headers = response['resultSet']['headers']
    season_data = pd.DataFrame(response['resultSet']['rowSet'], columns=table_headers)

    # Do we want this to be first or last?
    season_data['Year'] = season_year

    # Append the dataframe to the list
    dataframes.append(season_data)

# Concatenate all dataframes into one giant dataframe
player_stat_df = pd.concat(dataframes, ignore_index=True)
player_stat_df
#player_stat_df.to_csv('player_stats.csv')


**Team Stat Data Collection**
@Akhil add in process here

In [10]:
season_years = [
    "2003", "2004", "2005", "2006", "2007",
    "2008", "2009", "2010", "2012", "2013", "2014",
    "2015", "2016", "2017", "2018", "2019", "2020",
    "2021", "2022"
]

all_data = []

for season in season_years:
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}.html"

    result = requests.get(url)
    if result.status_code != 200:
        print("Something went wrong:", result.status_code, result.reason)
        continue

    page = BeautifulSoup(result.text, "html.parser")

    team_name_lst = page.find_all("th", {'data-stat': 'team_name'})
    team_names = []

    for x in team_name_lst:
        team_names.append(x.text)

    team_names.remove("Eastern Conference")
    team_names.remove("Western Conference")

    lst = ['wins', 'losses', 'win_loss_pct', 'pts_per_g', 'opp_pts_per_g', 'srs']
    result = []

    for x in lst:
        tmp = page.find_all("td", {'data-stat': f'{x}'})
        tmp_lst = []

        for y in tmp:
            if len(tmp_lst) >= len(team_names):
                break
            tmp_lst.append(y.text)

        result.extend(tmp_lst)


    season_data = []
    for team, wins, losses, w_l, avg_pts, opp_avg_pts, srs in zip(
            team_names, result[0::6], result[1::6], result[2::6], result[3::6], result[4::6], result[5::6]):
        season_data.append({
            'Season': season,
            'Team': team,
            'Wins': wins,
            'Losses': losses,
            'Win/Loss %': w_l,
            'Avg Pts': avg_pts,
            'Opp Avg Pts': opp_avg_pts,
            'SRS': srs
        })

    
    all_data.extend(season_data)


team_stats_df = pd.DataFrame(all_data)

# Save the combined data to a CSV file
#team_stats_df.to_csv('nba_team_data.csv', index=False)




**Historical MVP Data Collection**
This is the data historically ranking the MVPs over the seasons that are being analyzed. It goes through the list of target seasons and splits the text on the page to create a dataframe. This had to be done by using a text representation since the other sources were not able to be scraped.

In [12]:
url = "https://www.nba.com/news/history-mvp-award-winners"

response = requests.get(url)

if response.status_code != 200:
        print("Something went wrong:", response.status_code, response.reason)

page = BeautifulSoup(response.content, 'html.parser')


target_seasons = ["2003-04", "2004-05", "2005-06", "2006-07", "2007-08", "2008-09", "2009-10", \
    "2010-11", "2011-12", "2012-13", "2013-14", "2014-15", "2015-16", "2016-17", "2017-18", \
    "2018-19", "2019-20", "2020-21", "2021-22", "2022-23"]

data = {} 

# Loop through the target seasons
for target_season in target_seasons:
    for p_tag in page.find_all('p'):
        if target_season in p_tag.text:
            split_data = p_tag.text.split(' — ')
            if len(split_data) > 1:
                winner = split_data[1].split(',')[0]
                data[target_season] = winner

# Create a DataFrame from the data dictionary
mvp_df = pd.DataFrame(list(data.items()), columns=["Year", "MVP_Name"])
mvp_df
#mvp_df.to_csv('mvp_historical.csv')